In [ ]:
import cv2
import datetime
import math
import os
from pytube import YouTube

In [ ]:
url ='https://www.youtube.com/watch?v=Q3L_RlHFhIA'
yt = YouTube(url)
downloadPath = './Videos'

In [ ]:
for e in yt.streams:
    print(e)

In [ ]:
# Download in 720p 30fps
yt.streams.get_by_itag(136).download(downloadPath)

In [ ]:
# https://towardsdatascience.com/the-easiest-way-to-download-youtube-videos-using-python-2640958318ab

class FrameExtractor():
    '''
    Class used for extracting frames from a video file.
    '''
    def __init__(self, video_path):
        self.video_path = video_path
        self.vid_cap = cv2.VideoCapture(video_path)
        self.n_frames = int(self.vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.fps = int(self.vid_cap.get(cv2.CAP_PROP_FPS))
        
    def get_video_duration(self):
        duration = self.n_frames/self.fps
        print(f'Duration: {datetime.timedelta(seconds=duration)}')
        
    def get_n_images(self, every_x_frame):
        n_images = math.floor(self.n_frames / every_x_frame) + 1
        print(f'Extracting every {every_x_frame} (nd/rd/th) frame would result in {n_images} images.')
        
    def extract_frames(self, every_x_frame, img_name, delim='', dest_path=None, img_ext = '.jpg',get_n_minutes=0, skip_frames=0):
        n_images = math.floor(self.n_frames / every_x_frame) + 1
        
        img_cutoff = 0
        if get_n_minutes != 0:
            img_cutoff = (get_n_minutes*((self.fps/every_x_frame)*60))
        
        if not self.vid_cap.isOpened():
            self.vid_cap = cv2.VideoCapture(self.video_path)
        
        if dest_path is None:
            dest_path = os.getcwd()
        else:
            if not os.path.isdir(dest_path):
                os.mkdir(dest_path)
                print(f'Created the following directory: {dest_path}')
        
        frame_cnt = 0
        img_cnt = skip_frames

        while self.vid_cap.isOpened():
            if frame_cnt % 500 == 0:
                print(frame_cnt,skip_frames)
            
            success,image = self.vid_cap.read()
            
            if not success:
                break
            
            if frame_cnt >= skip_frames:
                if frame_cnt % every_x_frame == 0:
                    img_path = dest_path +'/'+ ''.join([img_name, delim, str(img_cnt), img_ext])
                    cv2.imwrite(img_path, image)  

                    img_cnt += 1
                    if img_cutoff != 0:
                        if img_cnt >= img_cutoff:
                            break
            
            frame_cnt += 1
        
        self.vid_cap.release()
        cv2.destroyAllWindows()
        print('100% done')

In [ ]:
import pathlib
path = pathlib.Path(downloadPath)
p = path.glob('*.mp4')
fullPath = ''
for i in p:
    fullPath = downloadPath +'/'+ str(i).split('/')[-1]
print(fullPath)

In [ ]:
fe = FrameExtractor(fullPath)

In [ ]:
framesPath = downloadPath+'/frames'

if not os.path.isdir(framesPath):
    os.mkdir(framesPath)

fe.extract_frames(every_x_frame=1,img_name='',dest_path=framesPath,get_n_minutes=4,skip_frames=0)